In [13]:
#-*- coding: utf-8 -*-
from sklearn.externals import joblib

In [14]:
clf = joblib.load('classify.model')
cate_dict = joblib.load('cate_dict.dat')
vectorizer = joblib.load('vectorizer.dat')

In [15]:
joblib.dump(clf, 'n_classify.model')

['n_classify.model',
 'n_classify.model_01.npy',
 'n_classify.model_02.npy',
 'n_classify.model_03.npy']

In [16]:
joblib.dump(cate_dict, 'n_cate_dict.dat')
joblib.dump(vectorizer, 'n_vectorizer.dat')

['n_vectorizer.dat']

In [17]:
cate_id_name_dict = dict(map(lambda (k,v):(v,k),cate_dict.items()))

In [18]:
print vectorizer.transform(['인텔 코어 i3 아이비브릿지 3220'])[0]

In [19]:
pred = clf.predict(vectorizer.transform(['인텔 코어 i3 아이비브릿지 3220']))[0]
print cate_id_name_dict[pred]

In [20]:
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

In [21]:
import MeCab
import urllib2
import json

from pandas import Series, DataFrame
import pandas as pd

In [22]:
m = MeCab.Tagger('-d /usr/lib/mecab/dic/mecab-ko-dic')

remove = ['SF', 'SE', 'SSO', 'SSC', 'SC', 'SY']
stop_words = ['신한카드', '삼성카드', 'BC카드', '국민카드']
count = 0

test_df = pd.read_pickle('url-image.df')

url_list = test_df.url
cate_list = test_df.image

In [23]:
from bottle import route, run, template,request,get,post

import time
from threading import Condition
_CONDITION = Condition()
@route('/classify')
def classify():
    
    global count
    count+=1
    print count
    
    print "classify called"
    img = request.GET.get('img', '')
    name = request.GET.get('name', '')
    
    category = ''
    
    for i in range(len(url_list)):
        if(url_list[i] == img):
            category = cate_list[i]

    real_name = str(unicode(name +' '+ category))
    
    parsed = m.parse(real_name.encode('utf-8'))
    entire_list = parsed.split('\n')
    
    mid_list = []
    new_d_elements = ''
    
    for element in entire_list:
        if(element == 'EOS' or len(element) < 2) :
            continue
        splited = element.split('\t')
        mid_list.append([splited[0], splited[1].split(',')])
            
    for element in mid_list:
        if element[1][0] in remove:
            continue
        if element[0] in stop_words:
            continue
        new_d_elements = new_d_elements+element[0]+' '
        
    #ret_value = json.load(urllib2.urlopen('http://192.168.99.100:5000/classify_url?imageurl='+img))
    #new_d_elements = new_d_elements+' '+ret_value['result'][1][0][0]
    
    pred = clf.predict(vectorizer.transform([new_d_elements]))[0]
    return {'cate': cate_id_name_dict[pred]}


run(host='0.0.0.0', port=8887)